In [7]:
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [8]:
for x in range(2013,2021):
    f = open(f"nov{x}-{x+1}.txt", "r")
    content = f.read()
    f.close()

    content = content.replace(" ","")

    g = open(f"nov{x}-{x+1}.txt", "w")
    g.write(content)
    g.close()

FileNotFoundError: [Errno 2] No such file or directory: 'nov2013-2014.txt'

In [18]:
frames = []

for x in range(2013,2021):
    frames.append(pd.read_csv(f"nov{x}-{x+1}.txt", sep = ","))

data = pd.concat(frames)

In [19]:
data.sample(10)

,Aeg,Temperatuur,Niiskus,&Otilde;hur&otilde;hk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus
33178,2016-03-0903:55:00,0.079888,99.017539,1011.523767,4.459584,280.394901,0.0,NaN,NaN,0.945066,NaN
36636,2019-03-1706:40:00,1.182063,94.838762,1000.952567,2.932020,227.492217,0.0,NaN,NaN,8.291684,NaN
80647,2017-08-2201:25:00,12.749678,97.998257,1014.336000,1.406290,171.808342,0.0,NaN,NaN,3.583456,NaN
15578,2019-01-0303:50:00,-4.515746,87.260668,1010.457667,5.543690,14.125565,0.0,NaN,NaN,5.534098,NaN
72410,2017-07-2411:00:00,20.013737,52.259113,1011.462467,1.426486,322.006329,0.0,NaN,NaN,722.427457,NaN
75236,2018-08-0306:35:00,20.369472,72.590028,1017.204367,1.480112,235.717097,0.0,NaN,NaN,111.654242,NaN
44950,2020-04-1610:05:00,5.820050,82.500302,984.473422,10.847912,286.278559,0.0,NaN,NaN,53.349942,NaN
78863,2020-08-1204:10:00,12.186425,85.838333,1022.580233,0.873457,326.402415,0.0,NaN,NaN,0.000000,NaN
86901,2014-11-1212:55:00,8.699358,88.031754,1001.923024,1.971105,109.214532,0.0,NaN,NaN,NaN,NaN
20320,2014-01-2414:00:00,-14.524525,54.176541,1026.154245,0.755851,322.444351,0.0,NaN,NaN,NaN,0.077813


In [20]:
data.shape

(811086, 11)

In [21]:
#f = open("kiirgusvoog_5a.csv", "r")
#content = f.read()
#f.close()

#content = content.replace(" ","")

#g = open("kiirgusvoog_5a_2.csv", "w")
#g.write(content)
#g.close()

#data = pd.read_csv("kiirgusvoog_5a_2.csv")

In [22]:
data = data.rename(columns = {'&Otilde;hur&otilde;hk':'Õhurõhk'})

In [23]:
def break_up_time(data):
    data["Aasta"]= data.apply(lambda row: row["Aeg"][0:4], axis=1)
    data["Kuu"] = data.apply(lambda row: row["Aeg"][5:7], axis=1)
    data["Päev"] = data.apply(lambda row: row["Aeg"][8:10], axis=1)
    data["Tund"] = data.apply(lambda row: row["Aeg"][10:12], axis=1)
    data["Minut"] = data.apply(lambda row: row["Aeg"][13:15], axis=1)
    data["Aeg"] = data.apply(lambda row: row["Aeg"][0:10], axis=1) + " " + data.apply(lambda row: row["Aeg"][10:18], axis=1)
    data.Aasta = data.Aasta.astype('int')
    data.Kuu = data.Kuu.astype('int')
    data.Päev = data.Päev.astype('int')
    data.Tund = data.Tund.astype('int')
    data.Minut = data.Minut.astype('int')

In [24]:
break_up_time(data)
data.head()

,Aeg,Temperatuur,Niiskus,Õhurõhk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus,Aasta,Kuu,Päev,Tund,Minut
0,2013-11-15 00:00:00,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,NaN,NaN,NaN,0.097777,2013,11,15,0,0
1,2013-11-15 00:05:00,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,NaN,NaN,NaN,0.134524,2013,11,15,0,5
2,2013-11-15 00:10:00,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,NaN,NaN,NaN,0.091138,2013,11,15,0,10
3,2013-11-15 00:15:00,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,NaN,NaN,NaN,0.097796,2013,11,15,0,15
4,2013-11-15 00:20:00,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,NaN,NaN,NaN,0.096651,2013,11,15,0,20


In [25]:
data.columns

Index(['Aeg', 'Temperatuur', 'Niiskus', 'Õhurõhk', 'Tuulekiirus', 'Tuulesuund',
       'Sademed', 'UVindeks', 'Valgustatus', 'Kiirgusvoog', 'Radioaktiivsus',
       'Aasta', 'Kuu', 'Päev', 'Tund', 'Minut'],
      dtype='object')

In [36]:
from datetime import datetime

data["Aeg"] = data.apply(lambda row: datetime.strptime(row["Aeg"],"%Y-%m-%d %H:%M:%S"), axis=1)

In [37]:
data.dtypes

Aeg               datetime64[ns]
Temperatuur              float64
Niiskus                  float64
Õhurõhk                  float64
Tuulekiirus              float64
Tuulesuund               float64
Sademed                  float64
UVindeks                 float64
Valgustatus              float64
Kiirgusvoog              float64
Radioaktiivsus           float64
Aasta                      int64
Kuu                        int64
Päev                       int64
Tund                       int64
Minut                      int64
dtype: object

In [38]:
data = data[["Aeg","Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuulekiirus","Tuulesuund","Sademed","Kiirgusvoog"]] # "Sadanudlumi"
#data['Sadanudlumi'] = data['Sadanudlumi'].fillna(0)
#data.columns = ["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuule kiirus","Tuule suund","Sademed","Kiirgusvoog"]
data['Sademed'] = data['Sademed'].fillna(0)
data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
data['Tuulesuund'] = data['Tuulesuund'].fillna(method='backfill')
data['Temperatuur'] = data['Temperatuur'].fillna(method='backfill')
data['Niiskus'] = data['Niiskus'].fillna(method='backfill')
data['Õhurõhk'] = data['Õhurõhk'].fillna(method='backfill')
data['Kiirgusvoog'] = data['Kiirgusvoog'].fillna(method='backfill')
data["KiirgusvoogTulevikus"] = data["Kiirgusvoog"].shift(-1)
data = data.dropna()
data.isnull().sum()

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_30414/1898045760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sademed'] = data['Sademed'].fillna(0)
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_30414/1898045760.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_30414/1898045760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy

Aeg                     0
Aasta                   0
Kuu                     0
Päev                    0
Tund                    0
Minut                   0
Temperatuur             0
Niiskus                 0
Õhurõhk                 0
Tuulekiirus             0
Tuulesuund              0
Sademed                 0
Kiirgusvoog             0
KiirgusvoogTulevikus    0
dtype: int64

In [39]:
data = data.rename(columns = {
    'Aeg': 'timestamp',
    'Aasta':'year',
    'Kuu':'month',
    'Päev':'day',
    'Tund':'hour',
    'Minut':'minute',
    'Temperatuur':'temperature',
    'Niiskus':'humidity',
    'Õhurõhk':'atmospheric_pressure',
    'Tuulekiirus':'wind_speed',
    'Tuulesuund':'wind_direction',
    'Sademed':'precipitation',
    #'Sadanudlumi':'snow',
    'Kiirgusvoog':'radiation_flux',
    'KiirgusvoogTulevikus':'rad_flux_infuture'    
    })
data.columns

Index(['timestamp', 'year', 'month', 'day', 'hour', 'minute', 'temperature',
       'humidity', 'atmospheric_pressure', 'wind_speed', 'wind_direction',
       'precipitation', 'radiation_flux', 'rad_flux_infuture'],
      dtype='object')

In [40]:
data[data.humidity == data.humidity.max()]

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
51611,2014-05-13 11:00:00,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0
51611,2014-05-13 11:00:00,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0


In [41]:
data.describe()

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
count,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000
mean,2017.398950,6.494621,15.751352,11.481656,27.512518,6.825025,75.458473,1010.824081,3.258511,199.267411,0.004576,89.464947,89.464947
std,2.424143,3.526065,8.820120,6.926916,17.262782,8.905742,19.869568,11.040802,1.678265,91.412932,0.050455,181.339887,181.339887
min,2013.000000,1.000000,1.000000,0.000000,0.000000,-24.307660,0.000000,963.557524,0.000000,0.000000,0.000000,-2.625525,-2.625525
25%,2016.000000,3.000000,8.000000,5.000000,14.000000,0.635374,62.394290,1003.885773,2.053779,124.554193,0.000000,0.000000,0.000000
50%,2018.000000,6.000000,16.000000,11.000000,30.000000,5.843931,81.315327,1011.116467,3.025117,224.132079,0.000000,2.639460,2.639460
75%,2019.000000,10.000000,23.000000,17.000000,45.000000,13.633509,91.499743,1018.149000,4.240506,274.416061,0.000000,74.003317,74.003317
max,2021.000000,12.000000,31.000000,23.000000,59.000000,33.553939,117.297204,1045.714767,15.497177,360.000000,9.799000,1132.409778,1132.409778


In [42]:
data.sample(50)

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
100591,2018-10-30 07:30:00,2018,10,30,7,30,-0.466109,80.174615,1025.542233,7.957209,100.021230,0.000,6.302302,7.349411
4283,2013-11-29 20:55:00,2013,11,29,20,55,-0.922516,58.279472,998.950814,2.274549,312.049617,0.000,0.000000,0.000000
104687,2018-11-13 12:50:00,2018,11,13,12,50,5.642785,93.601823,1015.422333,4.996556,216.225187,0.067,10.951960,10.686510
83862,2020-08-29 12:45:00,2020,8,29,12,45,16.459045,63.232679,1011.142067,2.399198,146.376940,0.000,305.080190,279.611543
19390,2021-01-12 22:05:00,2021,1,12,22,5,-2.625578,91.528844,1004.303167,4.209238,125.502025,0.000,6.604353,4.607668
92253,2020-09-27 16:00:00,2020,9,27,16,0,20.059727,52.620820,1014.961333,5.719832,103.314282,0.000,270.627708,263.871729
74096,2021-07-21 21:55:00,2021,7,21,21,55,16.897500,48.605000,1014.543000,1.593161,302.864067,0.000,0.000000,0.000000
7383,2016-12-10 15:05:00,2016,12,10,15,5,-3.543101,79.102652,1006.422367,2.269708,317.425793,0.000,2.290250,1.423720
3421,2019-11-26 21:05:00,2019,11,26,21,5,-2.493468,92.363087,1011.673500,2.208577,231.766787,0.000,3.990797,3.936879
56585,2016-05-29 11:30:00,2016,5,29,11,30,18.229477,67.171543,1021.458233,2.698730,88.967781,0.000,956.410962,624.373733


In [43]:
data.head(30)

,timestamp,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
0,2013-11-15 00:00:00,2013,11,15,0,0,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,0.0,0.0
1,2013-11-15 00:05:00,2013,11,15,0,5,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,0.0,0.0
2,2013-11-15 00:10:00,2013,11,15,0,10,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,0.0,0.0
3,2013-11-15 00:15:00,2013,11,15,0,15,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,0.0,0.0
4,2013-11-15 00:20:00,2013,11,15,0,20,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,0.0,0.0
5,2013-11-15 00:25:00,2013,11,15,0,25,2.075500,81.316279,1012.867377,1.369254,221.732026,0.0,0.0,0.0
6,2013-11-15 00:30:00,2013,11,15,0,30,2.041885,81.697612,1012.920600,1.314422,223.314140,0.0,0.0,0.0
7,2013-11-15 00:35:00,2013,11,15,0,35,2.038886,82.163594,1013.180392,1.228393,221.865827,0.0,0.0,0.0
8,2013-11-15 00:40:00,2013,11,15,0,40,2.022422,82.593353,1013.001292,1.528448,219.857440,0.0,0.0,0.0
9,2013-11-15 00:45:00,2013,11,15,0,45,2.020750,83.062526,1012.941358,1.403501,223.718700,0.0,0.0,0.0


In [45]:
data.to_csv("data.csv",index=False)

## trying to get data directly from web page

In [18]:
#url = 'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D=2016&begin%5Bmon%5D=11&begin%5Bmday%5D=18&end%5Byear%5D=2021&end%5Bmon%5D=11&end%5Bmday%5D=18&9=1&12=1&10=1&15=1&16=1&14=1&snow_16=1&ok=+Esita+päring+'


def get_weather_data(start_year, start_month, start_day, end_year, end_month, end_day):
    url = f'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D={start_year}&begin%5Bmon%5D={start_month}&begin%5Bmday%5D={start_day}&end%5Byear%5D={end_year}&end%5Bmon%5D={end_month}&end%5Bmday%5D={end_day}&9=1&12=1&11=1&14=1&ok=+Esita+p%C3%A4ring+'
    c = pd.read_csv(url)
    #c.columns = ['Aeg', 'Temperatuur', 'Niiskus', 'Valgustatus', 'Kiirgusvoog']
    return c

In [19]:
train = get_weather_data(2011, 1, 1, 2015, 12, 12)

In [20]:
train.head()

,Aeg,Temperatuur,Niiskus,Valgustatus,Kiirgusvoog
0,2011-01-01 00:00:00,-6.936343679417071,98.6784171261325,1354.9408873569798,
1,2011-01-01 00:05:00,-6.9568330698942695,98.59701577484581,1029.54357837049,
2,2011-01-01 00:10:00,-6.97364936736915,98.42287783684701,1418.1783680055298,
3,2011-01-01 00:15:00,-6.94325190753387,98.49124327899729,1294.6821264335601,
4,2011-01-01 00:20:00,-6.86547316481287,98.64064185206159,1374.1625144454501,


In [21]:
train.isnull().sum()

Aeg             0
 Temperatuur    0
 Niiskus        0
 Valgustatus    0
 Kiirgusvoog    0
dtype: int64

# Solcast.com

In [2]:
data = pd.read_csv("Solcast_PT5M.csv")

In [3]:
data.sample(30)

,PeriodEnd,PeriodStart,Period,AirTemp,AlbedoDaily,Azimuth,CloudOpacity,DewpointTemp,Dhi,Dni,...,Ghi,GtiFixedTilt,GtiTracking,PrecipitableWater,RelativeHumidity,SnowDepth,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith
537384,2012-02-09T22:10:00Z,2012-02-09T22:05:00Z,PT5M,-4.7,0.18,40,0.0,-7.2,0,0,...,0,0,0,3.9,82.2,0.5,1039.3,69,3.1,137
627587,2012-12-19T03:05:00Z,2012-12-19T03:00:00Z,PT5M,4.1,0.13,-74,0.0,3.9,0,0,...,0,0,0,10.5,98.6,0.0,1020.5,91,0.2,132
272008,2009-08-02T11:30:00Z,2009-08-02T11:25:00Z,PT5M,17.5,0.16,-172,77.7,15.5,177,0,...,177,158,177,29.8,88.5,0.0,1010.2,295,5.2,34
957487,2016-02-07T14:45:00Z,2016-02-07T14:40:00Z,PT5M,8.9,0.13,138,0.0,2.7,57,588,...,190,490,159,9.9,64.9,0.0,998.7,220,9.3,77
221637,2009-02-08T13:55:00Z,2009-02-08T13:50:00Z,PT5M,3.0,0.15,150,62.3,0.4,106,0,...,106,94,106,7.5,83.0,0.0,1003.2,261,5.0,72
265458,2009-07-10T17:40:00Z,2009-07-10T17:35:00Z,PT5M,16.1,0.15,77,56.3,9.9,108,0,...,108,97,108,19.8,67.0,0.0,1014.5,294,6.3,71
665749,2013-04-30T15:15:00Z,2013-04-30T15:10:00Z,PT5M,13.1,0.15,111,0.0,3.1,123,730,...,522,578,869,10.3,50.7,0.0,1023.8,21,5.4,57
365464,2010-06-22T23:30:00Z,2010-06-22T23:25:00Z,PT5M,10.9,0.16,3,0.0,9.0,0,0,...,0,0,0,19.8,88.6,0.0,1023.8,64,2.2,104
2519,2007-01-09T18:05:00Z,2007-01-09T18:00:00Z,PT5M,12.5,0.16,101,30.9,7.9,0,0,...,0,0,0,17.6,73.5,0.0,1009.6,227,10.8,110
211676,2009-01-04T23:50:00Z,2009-01-04T23:45:00Z,PT5M,1.2,0.12,-2,97.0,0.9,0,0,...,0,0,0,11.3,98.1,1.3,1014.7,274,3.1,151
